In [1]:
%pip -q install pydantic pydantic_settings pydantic[email]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#1. Create a User Pydantic model with fields: name (str), age (int), email (str). Validate an instance.
#2. Add a validator to ensure age is between 18 and 99; trigger error on invalid data.
#3. Extend User with an optional field role: str = "member", and display the default.
#6.  Use a Pydantic List[User] to validate a list of user dictionaries.
#9. Add custom validation to ensure that email contains "@" using Pydantic @validator
#10.  Use Field() to add metadata to Pydantic fields, such as description, and print the metadata via .schema().

from typing import Optional, List
from pydantic import BaseModel, ValidationError, Field, field_validator
class User(BaseModel):
  name: str = Field(..., description="Name of the user", example="Ali")
  age: int = Field(ge=18, le=99, description="Age of the user",example=23)
  email: str = Field(..., description="Email of the user", example="Ali@example.com")
  role: Optional[str] = "member"
  @field_validator("email")
  def valid_email(cls, value):
    if "@" not in value:
      raise SyntaxError("Email must have an '@' sign.")
    return value

class UserList(BaseModel):
  users: List[User]

u1 = User(name='Ali', age=23, email="Ali@example.com")
print(u1.model_json_schema())
u2 = User(name="Ahmed", age=34 , email="Ahmed@example.com", role="Admin")
u3 = User(name="Usman", age=24, email="Usman@example.com")
users = UserList(users=[u1,u2,u3])
print(users.__dict__)
try:
  u = User(name="Aleena", age=14, email="Aleena@example.com")
  print(u)
except ValidationError as e:
  print("ValidationError: ", e)

{'properties': {'name': {'description': 'Name of the user', 'example': 'Ali', 'title': 'Name', 'type': 'string'}, 'age': {'description': 'Age of the user', 'example': 23, 'maximum': 99, 'minimum': 18, 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'Email of the user', 'example': 'Ali@example.com', 'title': 'Email', 'type': 'string'}, 'role': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': 'member', 'title': 'Role'}}, 'required': ['name', 'age', 'email'], 'title': 'User', 'type': 'object'}
{'users': [User(name='Ali', age=23, email='Ali@example.com', role='member'), User(name='Ahmed', age=34, email='Ahmed@example.com', role='Admin'), User(name='Usman', age=24, email='Usman@example.com', role='member')]}
ValidationError:  1 validation error for User
age
  Input should be greater than or equal to 18 [type=greater_than_equal, input_value=14, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than_equal


In [3]:
#7.  Define a Pydantic GenericModel Response[T] with data: T, success: bool, and test with T = User.
#8. Serialize a Response[User] to JSON and print.
from pydantic.generics import GenericModel
from typing import Generic, TypeVar
T = TypeVar("T")
class Response(BaseModel, Generic[T]):
  data: T
  success: bool
response = Response[User](data=u1, success=True)
print(f"JSON: {response.model_dump_json()}")

JSON: {"data":{"name":"Ali","age":23,"email":"Ali@example.com","role":"member"},"success":true}


c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.generics:GenericModel` has been moved to `pydantic.BaseModel`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [4]:
#4.  Create a generic Box[T] class storing item: T and a method get_item().
#5.  Instantiate Box[int] and Box[str] to demonstrate usage.

from typing import Generic, TypeVar
T = TypeVar('T')
class Box(Generic[T]):
  def __init__(self, item:T):
    self.item = item
  def get_item(self) -> T:
    return self.item
b1 = Box[str]("Ball")
print(b1.get_item())
b2 = Box[int](456)
print(b2.get_item())

Ball
456


In [7]:
#5.  Create a class Employee with attribute salary, then subclass Manager(Employee) and override display_salary() to add a bonus.
class Employee:
  def __init__(self, salary):
    self.salary = salary
  def display_salary(self):
    return f"Employee Salary: {self.salary}"
class Manager(Employee):
  def __init__(self, salary, bonus_per=10):
    super().__init__(salary)
    self.bonus_per = bonus_per
  def display_salary(self):
    bonus = self.salary * (self.bonus_per/100)
    final_salary = self.salary + bonus
    return f"Manager Salary (with {self.bonus_per}% bouns): {final_salary}"
e1 = Employee(15000)
print(e1.display_salary())
m1 = Manager(15000)
print(m1.display_salary())

Employee Salary: 15000
Manager Salary (with 10% bouns): 16500.0


In [13]:
class Bank:
    def __init__(self, balance):
        self._balance = balance
        self.__pin = "1234"
b1 = Bank(12200)
b1._balance
b1.__pin

AttributeError: 'Bank' object has no attribute '__pin'